## Step 1

Head over to [twitter's api](https://apps.twitter.com) and create an account. 

Click on the tab at the top to create a new app. Fill out all the information. Put your class website as the website twitter redirects you to. Add a path for your callback as your callback url. We will be using this later in the class. Mine is: 	https://peaceful-badlands-42479.herokuapp.com/callback/twitter

Once your app has been created, navigate to the "keys and access tokens" tab. Click the "create access token" button on the bottom of the page. This should give you all the keys you need. Leave this tab open. We will be needed it in step 2.

## Step 2

Navigate to [here](https://github.com/ryanmcgrath/twython). We need to download the twitter api library for Django.

Once installed, copy your **```Consumer Key, Consumer Secret, Access Token, and Access Token Secret```** from the twitter api page. These will be substituted in for the keys listed below in the same order. Look at the example below for more details.


```
#This
twitter = Twython(APP_KEY, APP_SECRET,OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

# Becomes this
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

```

Don't forget to add these variables to your .env file! If someone gets a hold of them they will be able to post tweets on your behalf. 

You can load your credentials into your settings file like so:

```
TWITTER_APP_KEY = os.environ.get('TWITTER_APP_KEY', '')
TWITTER_APP_SECRET = os.environ.get('TWITTER_APP_SECRET', '')
TWITTER_OAUTH_TOKEN = os.environ.get('TWITTER_OAUTH_TOKEN', '')
TWITTER_OAUTH_TOKEN_SECRET = os.environ.get('TWITTER_OAUTH_TOKEN_SECRET', '')
```

Then import the variables into your views.py:
```
from classproject.settings import TWITTER_APP_KEY, TWITTER_APP_SECRET,TWITTER_OAUTH_TOKEN,TWITTER_OAUTH_TOKEN_SECRET
```

## Step 3

In your console type

```
pip install django-dotenv
```

Change your manage.py to this:


```
#!/usr/bin/env python
import os
import sys
import dotenv

if __name__ == "__main__":
    os.environ.setdefault("DJANGO_SETTINGS_MODULE", "classproject.settings")
    try:
        from django.core.management import execute_from_command_line
    except ImportError:
        # The above import may fail for some other reason. Ensure that the
        # issue is really that Django is missing to avoid masking other
        # exceptions on Python 2.
        try:
            import django
        except ImportError:
            raise ImportError(
                "Couldn't import Django. Are you sure it's installed and "
                "available on your PYTHONPATH environment variable? Did you "
                "forget to activate a virtual environment?"
            )
        raise

    dotenv.read_dotenv()
    execute_from_command_line(sys.argv)
```

## Step 4

* Create a heirarchy as seen [here](https://docs.djangoproject.com/en/1.10/howto/custom-management-commands/)
* Add a file to your commands folder. Name it 'tweet_chron.py' and add the following:

```
from django.core.management.base import BaseCommand, CommandError
from website.models import TWITTERTWEET, TWITTERUSER
import os

TWITTER_APP_KEY = os.environ.get('TWITTER_APP_KEY', '')
TWITTER_APP_SECRET = os.environ.get('TWITTER_APP_SECRET', '')
TWITTER_OAUTH_TOKEN = os.environ.get('TWITTER_OAUTH_TOKEN', '')
TWITTER_OAUTH_TOKEN_SECRET = os.environ.get('TWITTER_OAUTH_TOKEN_SECRET', '')
```
```
class Command(BaseCommand):
    help = 'pull in a few tweets based on a query'

    def add_arguments(self, parser):
        parser.add_argument('query', nargs='+', type=int)

    def handle(self, *args, **options):
        for q in options['query']:
            try:
                # Do Query Logic Here
            except Exception as e:
                raise CommandError(e)

            self.stdout.write(self.style.SUCCESS('SuccessQ'))
```

## Step 5

Add the scheduler. Type in:

```
heroku addons:create scheduler:standard
```

Navigate to your Heroku app. Click on Overview > Heroku Scheduler > Add a new job.

Use the following as the command:

***```python manage.py tweet_chron 'trump'```***

Set the time to however frequently you'd like

## Step 6

Now you try. We need to save all the tweets for a certain query every hour. In your tweet_chron.py file, use Twython to query the twitter api and return results based on whatever was sent through the 'query' variable. Save your values into a model. Make sure to restrict your results in some way. Your database only has 20MB of data, so let's not get too crazy with this lab.

Here is an example of how that's done:

```
TWITTERTWEET.objects.count() < 1000
```

Put the route to your website in the field below. Make sure to include the time that the tweet was sent. Here is [mine](https://peaceful-badlands-42479.herokuapp.com/site/twitter/recent/trump/).

### Hints:

* You're only going to be reading from your database in your views.py file
* You're only going to be inserting into your database in your tweet_chron.py file